In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from warnings import filterwarnings
import shutil

# todo: put back!!
#from DatasetsEvaluator.DatasetsEvaluator import DatasetsTester

import sys  
sys.path.insert(0, 'C:\python_projects\DatasetsEvaluator_project\DatasetsEvaluator')
import DatasetsEvaluator as de

filterwarnings('ignore')

In [2]:
cache_folder = "c:\\dataset_cache"
partial_result_folder = "c:\\intermediate_results"

In [3]:
# Generally we wish to delete the partial results folder in order to create it fresh each execution, unless
# this execution is continuing from a previous failure, such as power loss etc.
try:
    shutil.rmtree(partial_result_folder)
except:
    pass

## Example finding a single file

In [4]:
datasets_tester = de.DatasetsTester()
# todo: should be able to remove the de. now
matching_datasets = datasets_tester.find_by_name(['pol'], "classification")
matching_datasets

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
201,201,pol,1,1,active,ARFF,NaN,NaN,NaN,0.0,49.0,15000.0,0.0,0.0,49.0,0.0
722,722,pol,2,2,active,ARFF,9959.0,2.0,5041.0,2.0,49.0,15000.0,0.0,0.0,48.0,1.0


## Example collecting all datasets meeting some specified criteria

In [5]:
matching_datasets = datasets_tester.find_datasets( 
    problem_type = "classification",
    min_num_classes = 2,
    max_num_classes = 20,
    min_num_minority_class = 5,
    max_num_minority_class = np.inf,
    min_num_features = 0,
    max_num_features = np.inf,
    min_num_instances = 500,
    max_num_instances = 5_000,
    min_num_numeric_features = 2,
    max_num_numeric_features = 50,
    min_num_categorical_features=0,
    max_num_categorical_features=50)

print("Number matching datasets found:", len(matching_datasets))
display(matching_datasets.head())

Number matching datasets found: 237


,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
11,11,balance-scale,1,1,active,ARFF,288.0,3.0,49.0,3.0,5.0,625.0,0.0,0.0,4.0,1.0
15,15,breast-w,1,1,active,ARFF,458.0,2.0,241.0,2.0,10.0,699.0,16.0,16.0,9.0,1.0
18,18,mfeat-morphological,1,1,active,ARFF,200.0,10.0,200.0,10.0,7.0,2000.0,0.0,0.0,6.0,1.0
22,22,mfeat-zernike,1,1,active,ARFF,200.0,10.0,200.0,10.0,48.0,2000.0,0.0,0.0,47.0,1.0


## Example collecting the datasets specified above and running classification tests

In [6]:
# After viewing the matching datasets, it's possible to collect all, or some subset of these. The following
# code collects 5 matching datasets.
# Note: some datasets may have errors loading. 
# Note: As this uses the default False for keep_duplicated_names, some datasets may be removed.
datasets_tester.collect_data(max_num_datasets_used=5, method_pick_sets='pick_first', preview_data=False) 

# The following code undoes the previous collection and collects all matching datasets. 
# This is currently commented out, as it takes longer to execute. 
# datasets_tester.collect_data(max_num_datasets_used=-1, preview_data=False)

dt_1 = tree.DecisionTreeClassifier(min_samples_split=50, max_depth=6, random_state=0)
dt_2 = tree.DecisionTreeClassifier(min_samples_split=25, max_depth=5, random_state=0)
knn_1 = KNeighborsClassifier(n_neighbors=5)
knn_2 = KNeighborsClassifier(n_neighbors=10)

summary_df = datasets_tester.run_tests(estimators_arr = [
        ("Decision Tree", "Original Features", "min_samples_split=50, max_depth=6", dt_1),
        ("Decision Tree", "Original Features", "min_samples_split=25, max_depth=5", dt_2),
        ("kNN", "Original Features", "n_neighbors=5", knn_1),
        ("kNN", "Original Features", "n_neighbors=10", knn_2)]) 

display(summary_df)

Loading dataset from openml: 0, id: 15, name: breast-w
Loading dataset from openml: 1, id: 29, name: credit-approval
Loading dataset from openml: 2, id: 31, name: credit-g
Loading dataset from openml: 3, id: 37, name: diabetes
Loading dataset from openml: 4, id: 38, name: sick

Running test on 5 datastets
Running tests on dataset index: 0, dataset: breast-w
	Running tests with model: Decision Tree (Original Features), (min_samples_split=50, max_depth=6)
	Running tests with model: Decision Tree (Original Features), (min_samples_split=25, max_depth=5)
	Running tests with model: kNN (Original Features), (n_neighbors=5)
	Running tests with model: kNN (Original Features), (n_neighbors=10)
Running tests on dataset index: 1, dataset: credit-approval
	Running tests with model: Decision Tree (Original Features), (min_samples_split=50, max_depth=6)
	Running tests with model: Decision Tree (Original Features), (min_samples_split=25, max_depth=5)
	Running tests with model: kNN (Original Features),

,Dataset Index,Dataset,Dataset Version,Model,Feature Engineering Description,Hyperparameter Description,Avg f1_macro,Std dev between folds,Train-Test Gap,# Columns,Model Complexity,Fit Time
0,0,breast-w,1,Decision Tree,Original Features,"min_samples_split=50, max_depth=6",0.914181,0.027418,0.039338,9,18.2,0.003200
1,0,breast-w,1,Decision Tree,Original Features,"min_samples_split=25, max_depth=5",0.919330,0.048302,0.041924,9,20.2,0.003200
2,0,breast-w,1,kNN,Original Features,n_neighbors=5,0.960240,0.034650,0.015801,9,0.0,0.003200
3,0,breast-w,1,kNN,Original Features,n_neighbors=10,0.954913,0.044677,0.009826,9,0.0,0.003600
4,1,credit-approval,1,Decision Tree,Original Features,"min_samples_split=50, max_depth=6",0.812408,0.143264,0.080159,56,32.2,0.005800
5,1,credit-approval,1,Decision Tree,Original Features,"min_samples_split=25, max_depth=5",0.819447,0.123166,0.080589,56,31.4,0.005200
6,1,credit-approval,1,kNN,Original Features,n_neighbors=5,0.678681,0.047804,0.101645,56,0.0,0.003600
7,1,credit-approval,1,kNN,Original Features,n_neighbors=10,0.682375,0.036452,0.049373,56,0.0,0.002600
8,2,credit-g,1,Decision Tree,Original Features,"min_samples_split=50, max_depth=6",0.625053,0.050662,0.100991,76,39.0,0.006401
9,2,credit-g,1,Decision Tree,Original Features,"min_samples_split=25, max_depth=5",0.611930,0.036800,0.114782,76,38.6,0.006201


## Example collecting regression datasets and performing regression tests on these

In [7]:
datasets_tester = de.DatasetsTester()

# This example uses the default settings to select the datasets, then displays the results. 
# In the subsequent cell, we choose to collect a subset of these. 
matching_datasets = datasets_tester.find_datasets(problem_type = "regression",)
print("Number matching datasets found:", len(matching_datasets))
display(matching_datasets.head())

c:\users\aymke\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number matching datasets found: 500


,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
11,11,balance-scale,1,1,active,ARFF,288.0,3.0,49.0,3.0,5.0,625.0,0.0,0.0,4.0,1.0
15,15,breast-w,1,1,active,ARFF,458.0,2.0,241.0,2.0,10.0,699.0,16.0,16.0,9.0,1.0
18,18,mfeat-morphological,1,1,active,ARFF,200.0,10.0,200.0,10.0,7.0,2000.0,0.0,0.0,6.0,1.0


In [8]:
dt = tree.DecisionTreeRegressor(min_samples_split=50, max_depth=5, random_state=0)
knn = KNeighborsRegressor(n_neighbors=10)

datasets_tester.collect_data(max_num_datasets_used=10)

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("Decision Tree", "Original Features", "Default", dt),
                                        ("kNN", "Original Features", "Default", knn)],
                                       num_cv_folds=3,
                                       scoring_metric='r2',
                                       show_warnings=True) 

display(summary_df)

Loading dataset from openml: 0, id: 3, name: kr-vs-kp


c:\users\aymke\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


 Dataset is classification
Loading dataset from openml: 1, id: 15, name: breast-w
 Dataset is classification
Loading dataset from openml: 2, id: 29, name: credit-approval
 Dataset is classification
Loading dataset from openml: 3, id: 31, name: credit-g
 Dataset is classification
Loading dataset from openml: 4, id: 38, name: sick
 Dataset is classification
Loading dataset from openml: 5, id: 50, name: tic-tac-toe
 Dataset is classification
Loading dataset from openml: 6, id: 185, name: baseball
 Dataset is classification
Loading dataset from openml: 7, id: 311, name: oil_spill
 Dataset is classification
Loading dataset from openml: 8, id: 333, name: monks-problems-1
 Dataset is classification
Loading dataset from openml: 9, id: 334, name: monks-problems-2
 Dataset is classification
Loading dataset from openml: 10, id: 335, name: monks-problems-3
 Dataset is classification
Loading dataset from openml: 11, id: 451, name: irish
 Dataset is classification
Loading dataset from openml: 12, id

 Dataset is classification
Loading dataset from openml: 96, id: 1049, name: pc4
 Dataset is classification
Loading dataset from openml: 97, id: 1050, name: pc3
 Dataset is classification
Loading dataset from openml: 98, id: 1063, name: kc2
 Dataset is classification
Loading dataset from openml: 99, id: 1067, name: kc1
 Dataset is classification
Loading dataset from openml: 100, id: 1068, name: pc1
 Dataset is classification
Loading dataset from openml: 101, id: 1435, name: fourclass
Loading dataset from openml: 102, id: 1443, name: PizzaCutter1
 Dataset is classification
Loading dataset from openml: 103, id: 1444, name: PizzaCutter3
 Dataset is classification
Loading dataset from openml: 104, id: 1451, name: PieChart1
 Dataset is classification
Loading dataset from openml: 105, id: 1452, name: PieChart2
 Dataset is classification
Loading dataset from openml: 106, id: 1453, name: PieChart3
 Dataset is classification
Loading dataset from openml: 107, id: 1462, name: banknote-authenticati

,Dataset Index,Dataset,Dataset Version,Model,Feature Engineering Description,Hyperparameter Description,Avg r2,Std dev between folds,Train-Test Gap,# Columns,Model Complexity,Fit Time
0,0,analcatdata_hiroshima,1,Decision Tree,Original Features,Default,-7.192390,7.092208,7.534345,3,9.666667,0.002667
1,0,analcatdata_hiroshima,1,kNN,Original Features,Default,-4.100821,2.759921,4.153182,3,0.000000,0.002000
2,1,cps_85_wages,1,Decision Tree,Original Features,Default,0.012019,0.073634,0.353659,30,21.000000,0.002333
3,1,cps_85_wages,1,kNN,Original Features,Default,0.017388,0.049038,0.285105,30,0.000000,0.001333
4,2,chscase_foot,1,Decision Tree,Original Features,Default,-1.173231,2.032095,1.227189,35,15.000000,0.002333
5,2,chscase_foot,1,kNN,Original Features,Default,-1.805111,3.126543,1.891862,35,0.000000,0.001999
6,3,SWD,1,Decision Tree,Original Features,Default,0.372545,0.012749,0.088916,10,38.333333,0.002333
7,3,SWD,1,kNN,Original Features,Default,0.370619,0.045895,0.102530,10,0.000000,0.003000
8,4,LEV,1,Decision Tree,Original Features,Default,0.466828,0.051950,0.089334,4,38.333333,0.003000
9,4,LEV,1,kNN,Original Features,Default,0.509897,0.056924,0.103255,4,0.000000,0.002333


## Example wrting to and reading from local cache

In [ ]:
# This will read from openml.org
datasets_tester.collect_data(max_num_datasets_used=10, preview_data=False, save_local_cache=True, path_local_cache=cache_folder)

# This will read from the local cache
datasets_tester.collect_data(max_num_datasets_used=10, preview_data=False, check_local_cache=True, path_local_cache=cache_folder)

c:\users\aymke\appdata\local\programs\python\python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading dataset from openml: 0, id: 3, name: kr-vs-kp
 Dataset is classification
Loading dataset from openml: 1, id: 15, name: breast-w
 Dataset is classification
Loading dataset from openml: 2, id: 29, name: credit-approval
 Dataset is classification
Loading dataset from openml: 3, id: 31, name: credit-g
 Dataset is classification
Loading dataset from openml: 4, id: 38, name: sick
 Dataset is classification
Loading dataset from openml: 5, id: 50, name: tic-tac-toe
 Dataset is classification
Loading dataset from openml: 6, id: 185, name: baseball
 Dataset is classification
Loading dataset from openml: 7, id: 311, name: oil_spill
 Dataset is classification
Loading dataset from openml: 8, id: 333, name: monks-problems-1
 Dataset is classification
Loading dataset from openml: 9, id: 334, name: monks-problems-2
 Dataset is classification
Loading dataset from openml: 10, id: 335, name: monks-problems-3
 Dataset is classification
Loading dataset from openml: 11, id: 451, name: irish
 Dataset

Loading dataset from openml: 95, id: 1037, name: ada_prior
 Dataset is classification
Loading dataset from openml: 96, id: 1049, name: pc4
 Dataset is classification
Loading dataset from openml: 97, id: 1050, name: pc3
 Dataset is classification
Loading dataset from openml: 98, id: 1063, name: kc2
 Dataset is classification
Loading dataset from openml: 99, id: 1067, name: kc1


## Example Comparing Two Pipelines

In [ ]:
datasets_tester = de.DatasetsTester()
matching_datasets = datasets_tester.find_by_name(['arsenic-male-bladder'], "classification")
datasets_tester.collect_data() 

pipe1 = Pipeline([('scaler', MinMaxScaler()), ('knn_classifier', KNeighborsClassifier())])
pipe2 = Pipeline([('scaler', StandardScaler()), ('knn_classifier', KNeighborsClassifier())])

# This provides an example using some non-default parameters. 
summary_df = datasets_tester.run_tests(estimators_arr = [
                                        ("kNN with MinMaxScaler", "Original Features", "Default", pipe1),
                                        ("kNN with StandardScaler", "Original Features", "Default", pipe2)],
                                       num_cv_folds=3,
                                       show_warnings=True) 

display(summary_df)

## Example Resuming after a failure

In [ ]:
# run_tests() has the ability to save intermediate results to a folder and continue after a failure from a given
# start point. This example simulates this by setting ending_point in the first call to run_tests() and
# setting starting_point in the 2nd call. 

try:
    shutil.rmtree(partial_result_folder)
except:
    pass

datasets_tester = de.DatasetsTester()
matching_datasets = datasets_tester.find_by_name(['pol', 'credit-g'], "classification")

datasets_tester.collect_data(save_local_cache=True, check_local_cache=True, path_local_cache=cache_folder) 
ds = datasets_tester.get_dataset_collection()

summary_df = datasets_tester.run_tests(estimators_arr = [("dt", "Original Features", "max_depth=2", tree.DecisionTreeClassifier(max_depth=2)),
                                                         ("dt", "Original Features", "max_depth=3", tree.DecisionTreeClassifier(max_depth=3))],
                                       num_cv_folds=3,
                                       show_warnings=True,
                                       ending_point=1,
                                       partial_result_folder=partial_result_folder)

summary_df = datasets_tester.run_tests(estimators_arr = [("dt", "Original Features", "max_depth=2", tree.DecisionTreeClassifier(max_depth=2)),
                                                         ("dt", "Original Features", "max_depth=3", tree.DecisionTreeClassifier(max_depth=3))],
                                       num_cv_folds=3,
                                       show_warnings=True,
                                       starting_point=1,
                                       partial_result_folder=partial_result_folder)

summary_df